In [1]:
from collections import defaultdict
import os
import requests
import pandas as pd
from sqlalchemy import create_engine

In [2]:
def get_postal(only_postal:bool=True)->pd.DataFrame:
    '''
    Simple function that returns DK postal codes for DAWA. If only_post is false,
    postal code name, kommune, latitude and longitude are returned
    
    how to use:
    df = get_postal(only_postal=False)
    '''

    post_data = defaultdict(list) 
    url = 'http://dawa.aws.dk/postnumre'
    
    with requests.Session() as httpx:
        r = httpx.get(url)
    
    assert r.ok, 'Connection Error'
  
    posts = r.json()

    for _, post in enumerate(posts):
        post_data['Postal'].append(post['nr'])
        post_data['Name'].append(post['navn'])
        post_data['Kommuner'].append(post['kommuner'][0]['navn'] if post['kommuner'] else None)
        post_data['Longitude'].append(post['visueltcenter'][0])
        post_data['Latitude'].append(post['visueltcenter'][1])
        # post_data['bbox'].append(post['bbox'])

    if only_postal:
        return  pd.DataFrame(post_data)[['Postal']]
    else:
        return pd.DataFrame(post_data)
  
            

In [3]:
df = get_postal(False)

In [4]:
df.sample(10)

,Postal,Name,Kommuner,Longitude,Latitude
528,2930,Klampenborg,Gentofte,12.621329,55.796100
929,7960,Karby,Morsø,8.564436,56.761321
403,1822,Frederiksberg C,Frederiksberg,12.546084,55.672683
325,1708,København V,København,12.557452,55.669621
597,4241,Vemmelev,Slagelse,11.263738,55.364142
706,5000,Odense C,Odense,10.391744,55.403021
331,1715,København V,København,12.560637,55.668704
498,2620,Albertslund,Ballerup,12.344776,55.686952
211,1452,København K,København,12.568295,55.679541
959,8444,Balle,Syddjurs,11.110358,56.204969


In [5]:
df.head()

,Postal,Name,Kommuner,Longitude,Latitude
0,1050,København K,København,12.586001,55.680652
1,1051,København K,København,12.590227,55.679838
2,1052,København K,København,12.589550,55.679067
3,1053,København K,København,12.590730,55.677986
4,1054,København K,København,12.589066,55.677635


In [6]:
df.dtypes

Postal        object
Name          object
Kommuner      object
Longitude    float64
Latitude     float64
dtype: object

In [7]:
CONNECTION_URI = f"postgresql://{os.getenv('POSTGRES_USER','danpra')}:{os.getenv('POSTGRES_PASSWORD', 'postgrespwd')}@postgres:5432/bolig_db"
TABLE_NAME = 'dk_postals'

In [8]:
engine = create_engine(CONNECTION_URI)

In [9]:
df.to_sql(TABLE_NAME,engine)

OperationalError: (psycopg2.OperationalError) could not translate host name "postgres" to address: Name or service not known

(Background on this error at: http://sqlalche.me/e/e3q8)

In [10]:
url = 'https://api.boliga.dk/api/v2/search/results'
params = dict(pageSize=100, zipCodes='2650', page=1)

In [11]:
r = requests.get(url, params=params)

In [12]:
data = r.json()
data['meta']

{'searchGuid': '1d3aa503-0367-4bcc-b4b8-3caaaa65878b',
 'totalCount': 171,
 'totalPages': 2,
 'pageIndex': 1,
 'pageSize': 100,
 'minPage': 1,
 'maxPage': 2}

In [13]:
# https://api.boliga.dk/api/v2/sold/search/results?searchTab=1&sort=date-d
            # https://api.boliga.dk/api/v2/frontpage/stats
            # https://api.boliga.dk/api/v2/sold/search/results?zipcodeFrom=2650&zipcodeTo=2650&street=&page=1&sort=date-d
            # https://api.boliga.dk/api/v2/search/results?pageSize=50&searchTab=0&searchArchive=true&includeds=1

In [15]:
df = pd.DataFrame(data['results']).head()

In [16]:
df.columns

Index(['id', 'latitude', 'longitude', 'propertyType',
       'priceChangePercentTotal', 'energyClass', 'openHouse', 'price',
       'selfsale', 'rooms', 'size', 'lotSize', 'floor', 'buildYear', 'city',
       'isForeclosure', 'isActive', 'municipality', 'zipCode', 'street',
       'squaremeterPrice', 'area', 'daysForSale', 'createdDate',
       'isPremiumAgent', 'images', 'net', 'exp', 'basementSize', 'inWatchlist',
       'views', 'agentRegId', 'domainId', 'guid', 'agentDisplayName',
       'groupKey', 'downPayment', 'itemType', 'dawaId'],
      dtype='object')

In [20]:
df.dtypes

id                           int64
latitude                   float64
longitude                  float64
propertyType                 int64
priceChangePercentTotal      int64
energyClass                 object
openHouse                   object
price                        int64
selfsale                      bool
rooms                      float64
size                         int64
lotSize                      int64
floor                      float64
buildYear                    int64
city                        object
isForeclosure                 bool
isActive                      bool
municipality                 int64
zipCode                      int64
street                      object
squaremeterPrice           float64
area                         int64
daysForSale                  int64
createdDate                 object
isPremiumAgent                bool
images                      object
net                          int64
exp                          int64
basementSize        

In [21]:
df.drop(columns=['images'], inplace=True)
df.dtypes

id                           int64
latitude                   float64
longitude                  float64
propertyType                 int64
priceChangePercentTotal      int64
energyClass                 object
openHouse                   object
price                        int64
selfsale                      bool
rooms                      float64
size                         int64
lotSize                      int64
floor                      float64
buildYear                    int64
city                        object
isForeclosure                 bool
isActive                      bool
municipality                 int64
zipCode                      int64
street                      object
squaremeterPrice           float64
area                         int64
daysForSale                  int64
createdDate                 object
isPremiumAgent                bool
net                          int64
exp                          int64
basementSize                 int64
inWatchlist         

In [24]:
from abc import ABC, abstractmethod
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
from requests import Session
import numpy as np
import pandas as pd

In [25]:
class Boliga(ABC):

    def __init__(self, url, headers=None):
        
        session = Session()

        self.BASE_URL = url
        

        if headers is None:
            headers = {'User-Agent': ('Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) '
                          'AppleWebKit/537.36 (KHTML, like Gecko) '
                          'Chrome/39.0.2171.95 Safari/537.36'),
                    'Content-Type': 'application/json'}

        session.headers.update(headers)

        self.session = session
        self.meta_data = None
        self.store =  pd.DataFrame()

    
    def __repr__(self):
        return f'{self.__class__.__name__}(API={repr(self.BASE_URL)})'

    
    @abstractmethod
    def get_page(self, *args, **kwargs):
        pass
    
    
    def get_pages(self, *args, **kwargs):
        pass

In [26]:
class BoligaRecent(Boliga):
    
    '''
    expects Base URL
    e.g. 
        url = 'https://api.boliga.dk/api/v2/search/results'      
    '''

    def get_page(self, page=1, pagesize=100, postal='2650',verbose=False):
        '''Gather Data From Boliga API
            page:int page number
            pagesize:int number of boligs in a page
            postal:str zip/postal code of a city
            verbose:bool print mining progress
        '''
        
        params = {'page':page,
                 'pageSize':pagesize,
                 'zipCodes':postal,}
        

        r = self.session.get(self.BASE_URL, params=params)

        if r.ok:
            data = r.json()
            
            self.store = self.store.append(
                    pd.DataFrame(data.get('results')), ignore_index=True)
            self.meta_data = data.get('meta')

        else:
            self.store
            
        if verbose:
            print(f'[+] Gathering data from page {page:}.{" ":>5}Found {len(self.store):>5} estates'
                 f'{" ":>3}Time {datetime.now().strftime("%d-%m-%Y %H:%M:%S")}')

        return self
    
    
    
    def get_pages(self, page=1, pagesize=100, postal='2650',verbose=False):
        '''
         Parallel Gathering Data From Boliga
            page:int page number to start
            verbose:bool print mining progress
        '''
        
        # Make the first call to get total number of pages for split call pagesize split
        
        self.get_page(page=page, pagesize=pagesize, postal=postal, verbose=verbose)
        total_pages = self.meta_data.get('totalPages')
        
        # since we got the first page, we can get the rest
        
        if page <= total_pages:
            page += 1
            
            with ThreadPoolExecutor(max_workers=total_pages) as executor:
                _ = [executor.submit(self.get_page, 
                                     page=split,
                                     pagesize=pagesize, 
                                     postal=postal,
                                    verbose=verbose)
                     for split in range(page, total_pages+1)]
        
        return self
                                

In [27]:
t = BoligaRecent(url='https://api.boliga.dk/api/v2/search/results')

In [28]:
t

BoligaRecent(API='https://api.boliga.dk/api/v2/search/results')

In [29]:
t.get_pages(postal='2600',verbose=True)

[+] Gathering data from page 1.     Found   100 estates   Time 02-05-2020 12:58:38
[+] Gathering data from page 2.     Found   110 estates   Time 02-05-2020 12:58:38


BoligaRecent(API='https://api.boliga.dk/api/v2/search/results')

In [30]:
t.store.shape

(110, 39)

In [31]:
t.meta_data

{'searchGuid': '6bfdf7b2-cebf-4bfc-a4d8-06968ad5a7ab',
 'totalCount': 110,
 'totalPages': 2,
 'pageIndex': 2,
 'pageSize': 100,
 'minPage': 1,
 'maxPage': 2}